In [ ]:
import codebase.web_scrape_functions as wsf
import codebase.analysis_functions as af
import codebase.match_data as match_data
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import utils
from utils import logger
import os
import logging
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import codebase.graphing_functions as gf
from codebase.settings import CAREERS
from codebase.settings import LABEL_DATA

if utils.check_if_ipython():
    logger.disabled = True

%load_ext autoreload
%autoreload 2
logger.handlers[1].setLevel(logging.INFO)
# logger.disabled = True
pd.get_option("display.max_columns")

First step is to get Kohli's career averges for all shots, then his averages for just specific shots and then we can find his most productive shot. The problem with searching things based off of specific shots strings only is that there is a veriety in the way that shots are described in the commentary. We would need a machine learning algorithm to work and recognize what these shots are. But first we need to create a training set which is a pain in the ass.

In [ ]:
KOHLI_ID = '253802'
ROOT_PLAYER_ID = '303669'
WILLIAMSON_PLAYER_ID = '277906'
SPD_SMITH_ID = '267192'

In [ ]:
kohli_matches = wsf.get_player_match_list(KOHLI_ID)

In [ ]:
kohli_innings = af.get_cricket_totals(KOHLI_ID, kohli_matches, _type='bat', by_innings=True, is_object_id=True, from_scorecards=False)

In [ ]:
kohli_comms = af.get_player_contributions(KOHLI_ID, kohli_matches, _type='bat', by_innings=True, is_object_id=True)

In [ ]:
kohli_comms_flat = [x for comms in kohli_comms for x in comms.commentTextItems.to_list()]

In [ ]:
def search_shots_in_comms(contributions, search_keywords, exlude_words=[], primary_keywords=[], threshold = 0.5):
    shots = []
    for i, comms in enumerate(contributions):
        innings = comms.commentTextItems.to_list()
        search = af.search_for_phrases(innings, keywords=search_keywords, exclude_words=exlude_words, primary_keywords=primary_keywords, threshold = threshold)
        shots.append(comms.iloc[[i for i,x in enumerate(search[0]) if x == 1]])
        
    return shots

In [ ]:
## Cover drives 
cover_drive_primary_kw = ['cover drive', 'cover-drive']

cover_drive_search_kw = [
    'drive',
    ('punch', 0.3),
    ('cover', 0.3),
    ('full', 0.2),
    ('wide', 0.2) ,
    'outside edge',
    ('reach', 0.3),
    'slip',
    'edge',
    'driving'
]
cover_drive_exlude_words = [
    'run out',
    'pull', 'flick',
    'bouncer', 'short ball', 'stays back',
    'backfoot', 'top edge', 'leave',
    'lets one go', 'easy leave', 'leaves the ball',
    'leg side', 'leading edge', 'leg ', 'cut', 'leg.',
    'on the pads', 'left alone', ('short', 0.25)
]
cover_drives_played = search_shots_in_comms(kohli_comms, cover_drive_search_kw, cover_drive_exlude_words, primary_keywords=cover_drive_primary_kw)

In [ ]:
cover_drive_primary_kw = ['cover drive', 'cover-drive']

cover_drive_search_kw = [
    'drive',
    ('punch', 0.3),
    ('cover', 0.3),
    ('off side', 0.2),
    ('full', 0.25),
    ('wide', 0.2) ,
    ('outside edge', 0.3),
    ('reach', 0.3),
    ('slip', 0.25),
    ('edge', 0.3),
    'driving',
    ('force', 0.2),
    ('push', 0.3),
    ('pitched up', 0.3),
    ('Overpitched', 0.2),
    ('dab', 0.2),
    ('outside off', 0.25)

]
cover_drive_exlude_words = [
    'run out',
    'pull', 'flick',
    'bouncer', 'short ball', 'stays back',
    'backfoot', 'top edge', 'leave', 'back foot', ('back', 0.2),
    'lets one go', 'easy leave', 'leaves the ball', 'goes back',
    'leg side', 'leading edge', ('leg ', 0.25), 'cut', ('leg', 0.25),
    'on the pads', 'left alone', ('short', 0.25), ('midwicket', 0.2), ('long-on', 0.25), ('long on', 0.25), ('back of a length', 0.25)
]



In [ ]:
ind, scores = af.search_for_phrases(kohli_comms_flat, cover_drive_search_kw, cover_drive_exlude_words, primary_keywords=cover_drive_primary_kw, threshold=0.49)
cover_drives_career = [kohli_comms_flat[i] for i,x in enumerate(ind) if x==1]
cover_drives_almost = [kohli_comms_flat[i] for i,x in enumerate(scores) if x < 0.49 and x > 0]

In [ ]:
print(len(cover_drives_career))
cover_drives_career

In [ ]:
print(len(cover_drives_almost))
cover_drives_almost[1000:]

In [ ]:
[x for comms in cover_drives_played for x in comms.commentTextItems.to_list()]

In [ ]:
cover_drive_averages, cover_drive_totals, cover_drive_stats = af.analyse_batting(cover_drives_played)

In [ ]:
cover_drive_averages

In [ ]:
cover_drive_totals

In [ ]:
flick_primary_kw = [
    'flick'
]
flick_search_kw = [
    'tuck',
    'clipped',
    'clip',
    ('pads', 0.3),
    ('pad ', 0.3),
    'on the pads',
    #('leg side', 0.25),
    ('leg', 0.3),
    ('square', 0.19),
    'glance'
    'off the pads',
    'off his pads',
    ('nudge', 0.3),
    ('pick', 0.3),
    ('straight', 0.3),
    #('square leg', 0.25),
    ('whip', 0.4),
    'on the legs',
    ('work', 0.25),
    ('midwicket', 0.3),
    ('on side', 0.25),
    ('across', 0.3)
]
flick_exlude_words = [
    'run out', 'bumper', 'sweep', 'swept',
    'pull', 'drive', 'cover-drive', ('cover', 0.3),
    'bouncer', 'short ball', 'stays back',
    'backfoot', 'top edge',
    'lets one go', 'easy leave', 'leaves the ball', 'padded away',
    ('off side', 0.25), 'cut', 'left alone', ('point', 0.25), ('defend', 0.3), ('push', 0.25)
]

flick_shots_career = search_shots_in_comms(kohli_comms, flick_search_kw, flick_exlude_words, flick_primary_kw, threshold=0.49)

In [ ]:
flick_averages, flick_totals, flick_stats = af.analyse_batting(flick_shots_career)

In [ ]:
flick_averages
#Flicks mostly produce singles from Kohli, there are almost as many ones as there are dots with flicks, so this is a genrally 'safe' shot for Kohli which is quite productive.

In [ ]:
flick_totals